In [1]:
# Laura Chapman
# Script for developing statistical tools for Glue
# Computes statistics for subsets as well as entire data using compute_statistic
# Changed from astropy tables to pandas dataframe

# Format data well in a popup using qt
# Color code by subset

In [2]:
# Basic code that imports glue and loads in and links the data

import sys
from glue.core.data_factories import load_data
from glue.core import DataCollection
from glue.core.link_helpers import LinkSame
from glue.app.qt.application import GlueApplication
from glue.viewers.image.qt import ImageViewer
from glue_vispy_viewers.volume.volume_viewer import VispyVolumeViewer

image_filename='w5.fits'
catalog_filename='w5_psc.vot'

#load 2 datasets from files
catalog = load_data(catalog_filename)
image = load_data(image_filename)

dc = DataCollection([catalog,image])

# link positional information
dc.add_link(LinkSame(catalog.id['RAJ2000'], image.id['Right Ascension']))
dc.add_link(LinkSame(catalog.id['DEJ2000'], image.id['Declination']))

#Create subset based on filament mask
ra_state=(image.id['Right Ascension'] > 44) & (image.id['Right Ascension'] < 46)
subset_group=dc.new_subset_group('RA_Selection',ra_state)
subset_group.style.color = '#0000FF'



In [3]:
import glue.utils.array as gua
import glue.core.data as gcd
from astropy.table import Table

In [4]:
# Constructs a pandas DataFrame instead of an astropy table

import pandas as pd
from pandas import DataFrame
import numpy as np

In [5]:
# Defines a subset and runs statistics using compute_statistic for the subset
# Saves data using a pandas dataframe called my_pandas_data

# Data that subset is pulled from
data = dc[0]

# Define state and subset
state1 = data.id['Jmag'] > 14
subset1 = data.new_subset(state1, label='Jmag > 14')

# Same arrays as with full data
mean_array = []
median_array = []
min_array = []
max_array = []
sum_array = []
name_array = []
tables = []

headings = ('mean', 'median', 'minimum', 'maximum', 'sum')

print(data.label, 'subset1')
print() 
for j in range (0, len(data.components)):
    name = data.components[j].label # Get the name of each component
    name_array.append(name) # add to the name array to build the table
    mean_array.append(data.compute_statistic('mean', subset1.components[j], subset_state=subset1.subset_state))
    median_array.append(data.compute_statistic('median', subset1.components[j], subset_state=subset1.subset_state))       
    min_array.append(data.compute_statistic('minimum', subset1.components[j], subset_state=subset1.subset_state))       
    max_array.append(data.compute_statistic('maximum', subset1.components[j], subset_state=subset1.subset_state))      
    sum_array.append(data.compute_statistic('sum', subset1.components[j], subset_state=subset1.subset_state))        
   
column_data = np.asarray([mean_array, median_array, min_array, max_array, sum_array]).transpose()

my_pandas_data = pd.DataFrame(column_data, index=name_array, columns=headings)

w5_psc subset1



In [46]:
from PyQt5.QtWidgets import QApplication, QComboBox, QWidget, QPushButton, QHBoxLayout, QTableView,QGroupBox, QDialog, QVBoxLayout, QLabel,QGridLayout
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import pyqtSlot, QVariant
from PyQt5 import QtCore
import sys

from PyQt5 import QtCore, QtWidgets, QtGui
Qt = QtCore.Qt
# from PyQt5.QtCore import QVariant
from qtpy.QtWidgets import (QApplication, QLabel, QWidget, QComboBox, QCheckBox,
                            QVBoxLayout, QPushButton, QGridLayout, QTableView) 

class AbsolutePositioningExample(QWidget):
    ''' An example of PySide absolute positioning; the main window
        inherits from QWidget, a convenient widget for an empty window. '''
    def __init__(self,dc):
        # Initialize the object as a QWidget
        QWidget.__init__(self)
        
        self.dc=dc
#         self.pressed_dictionary= {}
#         for i in range (len(self.dc)):
#             for j in range (len(self.dc[i].components)):
#                 self.pressed_dictionary[]


        self.toggle_dictionary = dict((dc.labels[i], np.zeros(len(dc[i].components))) for i in range(0, len(dc)))

        self.setMinimumSize(800, 600)
        self.setMaximumSize(800, 600)

        self.setWindowTitle('Statistics')
        
        self.choose_data = QComboBox(self)
        self.choose_data.addItems(["Choose a dataset"])
        self.choose_data.addItems(dc.labels)
        self.choose_data.model().item(0).setEnabled(False)
        self.choose_data.move(300, 5)
        self.choose_data.currentIndexChanged.connect(self.selectionchange)
        
        self.table = QTableView(self)
        self.table.setGeometry(50, 250, 700, 300) 


        self.horizontalGroupBox = QGroupBox(self)
        self.horizontalGroupBox.setGeometry(50, 50, 700, 200) 

        
    def clicked(self):
        button=self.sender()
        if isinstance(button, QPushButton):
            if button.isChecked():
                print(button.text()) 
                # see where in the components the label is the same as the push button
                index = np.where(np.asarray(dc[self.data_index].components).astype(str) == button.text())[0]
                
                print(index)
                self.toggle_dictionary[dc.labels[self.data_index]][index] = 1

            else:
                index = np.where(np.asarray(dc[self.data_index].components).astype(str) == button.text())[0]
                self.toggle_dictionary[dc.labels[self.data_index]][index] = 0
            print(self.toggle_dictionary)    
                
    def selectionchange(self, i):
        self.data_index = i-1
        
    
    
        try:
            self.component_layout
            
            # Check the states of the buttons and edit the toggle dict
            print(self.component_layout)
            
            
            while self.component_layout.count():
                # While it exists it deletes the first button
                item = self.component_layout.takeAt(0)
                widget = item.widget()
                widget.deleteLater()
        except:
            self.component_layout = QGridLayout()
            
        ncols=3
        nrows=int(np.round((len(self.dc[self.data_index].components)/3.)+0.5))

        i=0
        for col in range(0,nrows):
            for row in range(0,ncols):
                btnlabel=str(self.dc[self.data_index].components[i])
                btn=QPushButton(btnlabel)
                btn.setCheckable(True)
                btn.toggle()
                btn.clicked.connect(self.clicked)
                self.component_layout.addWidget(btn,col,row)
                
                if self.toggle_dictionary[dc.labels[self.data_index]][i] == 0:
                    btn.setChecked(False)
                else:
                    btn.setChecked(True)
                    
                i=i+1
                
                if i==len(dc[self.data_index].components):
                    break    
                
        self.horizontalGroupBox.setLayout(self.component_layout)
    
class pandasModel(QtCore.QAbstractTableModel):
    # Set up the data in a form that allows it to be added to qt widget
    def __init__(self, df, parent=None):
        QtCore.QAbstractTableModel.__init__(self, parent)
        self.data_frame = df
        super(pandasModel, self).__init__(parent)      

    def rowCount(self, parent=None):
        return len(self.data_frame.values)

    def columnCount(self, parent=None):
        return self.data_frame.columns.size

    def data(self, index, role=Qt.DisplayRole):
        if index.isValid():
            if role == Qt.DisplayRole:
                return QVariant(str(
                    self.data_frame.values[index.row()][index.column()]))
        return QVariant()
    
    def headerData(self, col, orientation, role):
        if orientation == Qt.Horizontal and role == Qt.DisplayRole:
            return QVariant(self.data_frame.columns[col])
        if orientation == Qt.Vertical and role == Qt.DisplayRole:
            return QVariant(self.data_frame.index[col])
        return QVariant()  
        
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print('QApplication instance already exists: %s' % str(app))
ex = AbsolutePositioningExample(dc)
ex.show()
sys.exit(app.exec_())

QApplication instance already exists: <PyQt5.QtWidgets.QApplication object at 0x1518a17948>
World 0
[1]
{'w5_psc': array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.]), 'w5': array([ 0.,  0.,  0.,  0.,  0.])}
DEJ2000
[4]
{'w5_psc': array([ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.]), 'w5': array([ 0.,  0.,  0.,  0.,  0.])}


SystemExit: 0

/Users/laurachapman/anaconda/envs/glueviz-dev/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [21]:
# Track which buttons are clicked or not clicked 
# Save this info as a dictionary?  key words are different data sets 

NameError: name 'sender' is not defined

In [27]:
dc.labels

['w5_psc', 'w5']

In [29]:
toggle_dictionary = dict((dc.labels[i], np.zeros(len(dc[i].components))) for i in range(0, len(dc)))

In [32]:
toggle_dictionary[dc.labels[1]][0]

0.0